# Препроцессинг диалоговых данных 

## В этом ноутбуке:

* Все вопросы-ответы (пара) заносятся в один датафрейм **raw_data** с указанием комитета (по возможности) и созыва Госдумы
* В датафрейм **processed_data** к raw_data добавлены косинусное расстояние между эмбеддингами вопроса и ответа, а также, как для вопроса, так и для ответа, доля того, что текст эмоционально-нейтральный, эмоционально-негативный, эмоционально-положительный. (а также информация о том, насколько модель уверена в своем результате, и какова доля "пустых" разговорных фраз)
* Датафрейм **processed_data_nameless** возвращает то же, что и *processed_data*, но для получения численных значений использует тексты без имен и отчеств (которые присутствуют практически в каждом диалоге в Думе).

In [23]:
!pip install torch >> None
!pip install transformers >> None
!pip install transformers sentencepiece >> None

!pip install Dostoevsky >> None
!python -m dostoevsky download fasttext-social-network-model >> None

!pip install natasha >> None

In [24]:
# библиотеки для работы с моделями платформы HuggingFace
import torch
from transformers import AutoTokenizer, AutoModel

# библиотеки для работы с эмбеддингами и текстами
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

# библиотеки для получения векторных представлений 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# библиотека Dostoevsky для задачи sentiment analysis
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

# Библиотека Natasha для получения именных представлений
# (named entity recognition)
from natasha import NamesExtractor
from natasha import MorphVocab

# библиотеки для загрузки данных
import os
import re
import glob
import requests
import ast
import pickle

# библиотеки для анализа данных
import numpy as np
import pandas as pd

# вспомогательные библиотеки
import warnings
# FutureWarnings предупреждения
warnings.filterwarnings("ignore", category=FutureWarning)

In [34]:
# Удалим данные, если они уже загружены локально

!rm -rf Scraping_stenograms

# Загрузим данные из подготовленного репозитория Guthub

! git clone https://github.com/AleksandraVasilieva/Scraping_stenograms/ 

Cloning into 'Scraping_stenograms'...
remote: Enumerating objects: 563, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 563 (delta 50), reused 129 (delta 34), pack-reused 390
Receiving objects: 100% (563/563), 4.65 MiB | 18.45 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [26]:
nltk.download('punkt')
nltk.download("stopwords")

# загрузка стоп слов, которые удаляются из текстов
stops = stopwords.words('russian')
# стеммер
stemmer = SnowballStemmer('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
def custom_tokenizer(x):
    # приведем все символы строки в нижний регистр и токенизируем строку: 
    #разобьем ее на отдельные слова
    step1 = word_tokenize(x)
    # уберем цифры и пунктуацию
    step2 = [i.lower() for i in step1 if i.isalpha()]
    # уберем стоп-слова
    step3 = [word for word in step2 if not word in stops]
    # проведем стемминг: убрем аффиксы
    step4 = [stemmer.stem(w) for w in step3]
    return step4

In [28]:
# Токенизатор и модель для Sentiment Analysis
dostoevsky_tokenizer = RegexTokenizer()
dostoevsky_sentiment_model = FastTextSocialNetworkModel(tokenizer=dostoevsky_tokenizer)
# results = dostoevsky_sentiment_model.predict([text])

# Токенизатор и модель для получения эмбеддингов
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
rubert_embeddings_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# Модель Наташа для удаления имен
morph_vocab = MorphVocab()
natasha_names_extractor = NamesExtractor(morph_vocab)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
# Функция для получения косинусного расстояния между эмбеддингами
def calculate_cosine_distance(embedding1, embedding2):
    distance = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))
    similarity = 1 - distance
    return similarity[0][0]

# Функция для получения эмбеддинга текста
def get_embeddings(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [30]:
# Функция для получения косинусного расстояния между двумя необработанными текстами
def get_cosine_similarity(question, answer):
  '''
  Функция принимает на вход вопрос и ответ,
  токенизирует, получает векторное представление
  (эмбеддинги) и возвращает косинусное расстояние.

  '''
  # Получение эмбеддингов
  embedding_question = get_embeddings(question, rubert_embeddings_model, rubert_tokenizer)
  embedding_answer = get_embeddings(answer, rubert_embeddings_model, rubert_tokenizer)

  # Получение косинусного расстояния

  cosine_distance = calculate_cosine_distance(embedding_question, embedding_answer)
  
  return cosine_distance

In [31]:
# Функция для удаления имен в строке
def nameless_text(text):
  '''
  Функция вернет текст без имен и отчеств
  '''

  # Найдем имена
  matches = list(natasha_names_extractor(text))

  # Извлечем имена из текста
  for match in matches:
      start = match.start
      end = match.stop
      text = text[:start] + text[end:]

  return text

In [35]:
# создадим датафрейм для сырого текста
raw_data = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Date', 'Question', 'Answer'])

# создадим датафрейм с добавочной информацией

processed_data = pd.DataFrame(columns=['Duma Convocation', 'Commitee name',  'Date', 'Question', 'Answer', \
        'Cosine Similarity', 'q_neutral', 'q_negative', 'q_positive', 'q_unsure', 'q_colloquial',
        'a_neutral', 'a_negative', 'a_positive', 'a_unsure', 'a_colloquial'])

root_path = '/content/Scraping_stenograms/dialogs data pickle format/'

# Пройдем по каждой папке
folders = ['duma 6/', 'duma 7/', 'duma 8/'] 
for folder in folders:

    session_name = folder[:-1]
    folder_link = root_path + folder

    # Узнаем, какие файлы в ней лежат
    files = glob.glob(folder_link + '/*')

    # Пройдем по каждому файл из текущей папки
    for path in files:

        commitee_name = path[len(path) - path[::-1].find('_'):][:-4]

        with open(path, 'rb') as file:
          file_content = pickle.load(file)


        # Пропустим файл, если он пустой
        if len(file_content) == 0:
          continue
          
        for chunk in file_content:

            if len(chunk) < 2:
              continue

            date_of_dialog = chunk[0][0]
            qa_session = chunk[1]

            if len(qa_session) < 2:
              continue

            # Будем считывать каждые 2 строки
            for index in range(0, len(qa_session), 2):
              question = qa_session[index].strip()
              answer = qa_session[index+1].strip()

              # вычислим косинусное расстояние между эмбеддингами
              cos_similarity = get_cosine_similarity(question, answer)

              # вычислим для вопроса и ответа их sentiment
              sentiment_dict_question = (dostoevsky_sentiment_model.predict([question]))[0]
              sentiment_dict_answer = (dostoevsky_sentiment_model.predict([answer]))[0]
              q_neutral = sentiment_dict_question['neutral']
              q_negative = sentiment_dict_question['negative']
              q_positive = sentiment_dict_question['positive']
              q_unsure = sentiment_dict_question['skip']
              q_colloquial = sentiment_dict_question['speech']
              a_neutral = sentiment_dict_answer['neutral']
              a_negative = sentiment_dict_answer['negative']
              a_positive = sentiment_dict_answer['positive']
              a_unsure = sentiment_dict_answer['skip']
              a_colloquial = sentiment_dict_answer['speech']


              # Добавим данные в датафрейм processed_data
              processed_data = processed_data.append({'Duma Convocation':session_name, \
              'Commitee name':commitee_name, 'Date': date_of_dialog,\
              'Question': question, \
              'Answer': answer,'Cosine Similarity' : cos_similarity,  \
              'q_neutral': q_neutral, 'q_negative': q_negative,  \
              'q_positive':q_positive, \
              'q_unsure':q_unsure, 'q_colloquial':q_colloquial,\
              'a_neutral':a_neutral, 'a_negative':a_negative, \
              'a_positive':a_positive, 'a_unsure':a_unsure, 
              'a_colloquial': a_colloquial},\
              ignore_index=True)
                
              # Добавим данные в датафрейм raw_data
              raw_data = raw_data.append({'Duma Convocation':session_name, \
                              'Commitee name':commitee_name,  'Date': date_of_dialog,\
                              'Question': question, 'Answer': answer}, ignore_index=True)
              



,Duma Convocation,Commitee name,Date,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, к вам вопрос.Мы все знаем, что как только начинается зима — море аварий и так далее, и все понимают, что на уровне ведомственных нормативных правовых актов много остаётся нерешённых вопросов.Вопрос: как, по вашему мнению, работа данного закона улучшит порядок расследования причин аварий, возникающих при теплоснабжении, и что надо сделать, чтобы всё-таки был контроль за ремонтом систем теплоснабжения?Спасибо за внимание.","Спасибо.Василий Михайлович, как уже сказал Юрий Александрович и я отметил, этот законопроект не является завершающим в нашей работе, мы продолжим совершенствование в части повышения контроля за надёжностью и качеством. Тем не менее конкретно эта норма чётко регламентирует полномочия правительства в части разработки порядка готовности субъектов теплоэнергетики к зиме. И в этом смысле то, на что вы обращаете внимание, точно улучшится. Но мы продолжим и дальше совершенствовать эту работу.",0.182020,0.651365,0.092698,0.053413,0.085109,0.060097,0.615098,0.115971,0.073706,0.067557,0.030225
1,duma 6,energ,20 февраля 2013 г,"Михаил Юрьевич, мой вопрос в продолжение вопроса моего коллеги Тарасюка. 18 февраля состоялся ""круглый стол"" на тему ""Федеральный закон ""О теплоснабжении"" — первые итоги применения"". Было отмечено, что более 40 тысяч километров тепловых сетей в нашей стране находятся в аварийном состоянии, а их изношенность составляет более 70 процентов. Так вот, в продолжение вопроса предыдущего оратора: не считаете ли вы, что федеральный орган исполнительной власти, осуществляющий функции по контролю и надзору в сфере безопасности электрических и тепловых установок и сетей, должен всё же больше уделять внимания профилактике предупреждения аварий — а мы понимаем, что в связи с такой высокой степенью изношенности количество аварий будет расти, — нежели всё же расследованию, зачастую запоздалому, причин аварийных ситуаций?Спасибо.","Дмитрий Иванович, порядок, о котором мы говорим, предполагает не только расследование аварий, но и проверку готовности, в данном случае муниципальных образований, к зиме в части систем теплоснабжения, к тому же и действующие нормативные правовые акты чётко регламентируют порядок подготовки, профилактику готовности к зиме, это то, о чём вы говорите. Тем не менее существующее положение дел требует совершенствования порядка. На упомянутом вами ""круглом столе"" нами, как Министерством энергетики, был предложен набор мер по улучшению состояния в этой сфере, мы, надеемся, в ближайшее время уже внесём это в качестве законодательной инициативы.",0.208041,0.705795,0.132974,0.038476,0.071601,0.033096,0.803184,0.228166,0.051855,0.075868,0.010997
2,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, законопроектом предлагается отнести к полномочиям федерального органа по контролю в сфере безопасности электрических и тепловых установок и сетей проверку готовности муниципальных образований к отопительному периоду, вы сейчас об этом сказали. Это безусловно позитивно, поскольку состояние муниципальных объектов теплоснабжения общеизвестно. Вопрос: не планирует ли министерство, помимо предложения о мерах контроля, внести в правительство предложение о разработке федеральной целевой программы реконструкции муниципальных объектов теплоснабжения в стране, чтобы кардинально решить проблему ежегодных зимних аварий, на ликвидацию которых затрачиваются огромные средства, учитывая, что всё-таки Минэнерго, ну, более профессионально в этом разбирается, чем Минрегион?","Спасибо, Виктор Андреевич и за вопрос, и за оценку работы Министерства энергетики.Как я уже сказал, на указанном ""круглом столе"" мы предложили всё-таки чуть более широкий спектр предложений, это требует отдельного доклада по поводу того, как улучшить ситуацию в данной сфере. Это не только и не стольк

In [36]:
#processed_data.sort_values(by = 'Cosine Similarity', ascending = False)
processed_data[processed_data['Duma Convocation'] == 'duma 6'][:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, к вам вопрос.Мы все знаем, что как только начинается зима — море аварий и так далее, и все понимают, что на уровне ведомственных нормативных правовых актов много остаётся нерешённых вопросов.Вопрос: как, по вашему мнению, работа данного закона улучшит порядок расследования причин аварий, возникающих при теплоснабжении, и что надо сделать, чтобы всё-таки был контроль за ремонтом систем теплоснабжения?Спасибо за внимание.","Спасибо.Василий Михайлович, как уже сказал Юрий Александрович и я отметил, этот законопроект не является завершающим в нашей работе, мы продолжим совершенствование в части повышения контроля за надёжностью и качеством. Тем не менее конкретно эта норма чётко регламентирует полномочия правительства в части разработки порядка готовности субъектов теплоэнергетики к зиме. И в этом смысле то, на что вы обращаете внимание, точно улучшится. Но мы продолжим и дальше совершенствовать эту работу.",0.18202,0.651365,0.092698,0.053413,0.085109,0.060097,0.615098,0.115971,0.073706,0.067557,0.030225


In [ ]:
#pd.set_option('display.max_colwidth', None)

In [ ]:
# Пример: считать ссылку с гитхаба

# url = 'https://github.com/AleksandraVasilieva/Scraping_stenograms/raw/main/dialogs%20data/session_6_all_dialogs_agrar.pkl'

# response = requests.get(url)
# import tempfile

# with tempfile.NamedTemporaryFile(delete=False) as temp_file:
#     temp_file.write(response.content)

#     # Access the filename using temp_file.name if needed

#     temp_file.seek(0)  # Reset file pointer to the beginning

#     loaded_list = pickle.load(temp_file)

# loaded_list[0]

In [38]:
raw_data[:1]

,Duma Convocation,Commitee name,Date,Question,Answer
0,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, к вам вопрос.Мы все знаем, что как только начинается зима — море аварий и так далее, и все понимают, что на уровне ведомственных нормативных правовых актов много остаётся нерешённых вопросов.Вопрос: как, по вашему мнению, работа данного закона улучшит порядок расследования причин аварий, возникающих при теплоснабжении, и что надо сделать, чтобы всё-таки был контроль за ремонтом систем теплоснабжения?Спасибо за внимание.","Спасибо.Василий Михайлович, как уже сказал Юрий Александрович и я отметил, этот законопроект не является завершающим в нашей работе, мы продолжим совершенствование в части повышения контроля за надёжностью и качеством. Тем не менее конкретно эта норма чётко регламентирует полномочия правительства в части разработки порядка готовности субъектов теплоэнергетики к зиме. И в этом смысле то, на что вы обращаете внимание, точно улучшится. Но мы продолжим и дальше совершенствовать эту работу."


In [39]:
processed_data[:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, к вам вопрос.Мы все знаем, что как только начинается зима — море аварий и так далее, и все понимают, что на уровне ведомственных нормативных правовых актов много остаётся нерешённых вопросов.Вопрос: как, по вашему мнению, работа данного закона улучшит порядок расследования причин аварий, возникающих при теплоснабжении, и что надо сделать, чтобы всё-таки был контроль за ремонтом систем теплоснабжения?Спасибо за внимание.","Спасибо.Василий Михайлович, как уже сказал Юрий Александрович и я отметил, этот законопроект не является завершающим в нашей работе, мы продолжим совершенствование в части повышения контроля за надёжностью и качеством. Тем не менее конкретно эта норма чётко регламентирует полномочия правительства в части разработки порядка готовности субъектов теплоэнергетики к зиме. И в этом смысле то, на что вы обращаете внимание, точно улучшится. Но мы продолжим и дальше совершенствовать эту работу.",0.18202,0.651365,0.092698,0.053413,0.085109,0.060097,0.615098,0.115971,0.073706,0.067557,0.030225


In [43]:
# создадим датафрейм с добавочной информацией, где из текстов убираются имена

processed_data_nameless = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Date', 'Question', 'Answer', \
        'Cosine Similarity', 'q_neutral', 'q_negative', 'q_positive', 'q_unsure', 'q_colloquial',
        'a_neutral', 'a_negative', 'a_positive', 'a_unsure', 'a_colloquial'])

root_path = '/content/Scraping_stenograms/dialogs data pickle format/'

# Пройдем по каждой папке
folders = ['duma 6/', 'duma 7/', 'duma 8/'] 
for folder in folders:

    session_name = folder[:-1]
    folder_link = root_path + folder

    # Узнаем, какие файлы в ней лежат
    files = glob.glob(folder_link + '/*')

    # Пройдем по каждому файл из текущей папки
    for path in files:

        commitee_name = path[len(path) - path[::-1].find('_'):][:-4]

        with open(path, 'rb') as file:
          file_content = pickle.load(file)

        # Пропустим файл, если он пустой
        if len(file_content) == 0:
          continue
          
        for chunk in file_content:

            if len(chunk) < 2:
              continue

            date_of_dialog = chunk[0][0]
            qa_session = chunk[1]

            if len(qa_session) < 2:
              continue

            # Будем считывать каждые 2 строки
            for index in range(0, len(qa_session), 2):
              question = qa_session[index].strip()
              answer = qa_session[index+1].strip()

              question_nameless = nameless_text(question)
              answer_nameless = nameless_text(answer)

              # вычислим косинусное расстояние между эмбеддингами
              cos_similarity = get_cosine_similarity(question_nameless, answer_nameless)

              # вычислим для вопроса и ответа их sentiment
              sentiment_dict_question = (dostoevsky_sentiment_model.predict([question_nameless]))[0]
              sentiment_dict_answer = (dostoevsky_sentiment_model.predict([answer_nameless]))[0]
              q_neutral = sentiment_dict_question['neutral']
              q_negative = sentiment_dict_question['negative']
              q_positive = sentiment_dict_question['positive']
              q_unsure = sentiment_dict_question['skip']
              q_colloquial = sentiment_dict_question['speech']
              a_neutral = sentiment_dict_answer['neutral']
              a_negative = sentiment_dict_answer['negative']
              a_positive = sentiment_dict_answer['positive']
              a_unsure = sentiment_dict_answer['skip']
              a_colloquial = sentiment_dict_answer['speech']


              # Добавим данные в датафрейм processed_data_nameless
              processed_data_nameless = processed_data_nameless.append({'Duma Convocation':session_name, \
              'Commitee name':commitee_name, 'Date': date_of_dialog,\
              'Question': question, \
              'Answer': answer,'Cosine Similarity' : cos_similarity,  \
              'q_neutral': q_neutral, 'q_negative': q_negative,  \
              'q_positive':q_positive, \
              'q_unsure':q_unsure, 'q_colloquial':q_colloquial,\
              'a_neutral':a_neutral, 'a_negative':a_negative, \
              'a_positive':a_positive, 'a_unsure':a_unsure, 
              'a_colloquial': a_colloquial},\
              ignore_index=True)


In [45]:
processed_data_nameless[:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, к вам вопрос.Мы все знаем, что как только начинается зима — море аварий и так далее, и все понимают, что на уровне ведомственных нормативных правовых актов много остаётся нерешённых вопросов.Вопрос: как, по вашему мнению, работа данного закона улучшит порядок расследования причин аварий, возникающих при теплоснабжении, и что надо сделать, чтобы всё-таки был контроль за ремонтом систем теплоснабжения?Спасибо за внимание.","Спасибо.Василий Михайлович, как уже сказал Юрий Александрович и я отметил, этот законопроект не является завершающим в нашей работе, мы продолжим совершенствование в части повышения контроля за надёжностью и качеством. Тем не менее конкретно эта норма чётко регламентирует полномочия правительства в части разработки порядка готовности субъектов теплоэнергетики к зиме. И в этом смысле то, на что вы обращаете внимание, точно улучшится. Но мы продолжим и дальше совершенствовать эту работу.",0.231715,0.47659,0.115971,0.047436,0.112805,0.075868,0.685959,0.100889,0.122533,0.098089,0.009136


In [46]:
processed_data_nameless[processed_data_nameless['Duma Convocation'] == 'duma 6'][:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,energ,20 февраля 2013 г,"Уважаемый Михаил Юрьевич, к вам вопрос.Мы все знаем, что как только начинается зима — море аварий и так далее, и все понимают, что на уровне ведомственных нормативных правовых актов много остаётся нерешённых вопросов.Вопрос: как, по вашему мнению, работа данного закона улучшит порядок расследования причин аварий, возникающих при теплоснабжении, и что надо сделать, чтобы всё-таки был контроль за ремонтом систем теплоснабжения?Спасибо за внимание.","Спасибо.Василий Михайлович, как уже сказал Юрий Александрович и я отметил, этот законопроект не является завершающим в нашей работе, мы продолжим совершенствование в части повышения контроля за надёжностью и качеством. Тем не менее конкретно эта норма чётко регламентирует полномочия правительства в части разработки порядка готовности субъектов теплоэнергетики к зиме. И в этом смысле то, на что вы обращаете внимание, точно улучшится. Но мы продолжим и дальше совершенствовать эту работу.",0.231715,0.47659,0.115971,0.047436,0.112805,0.075868,0.685959,0.100889,0.122533,0.098089,0.009136


In [47]:
# raw_data.to_csv('raw_data.csv', index=False)
# processed_data.to_csv('processed_data.csv', index=False)
# processed_data_nameless.to_csv('processed_data_nameless.csv', index=False)